In [1]:
import pandas as pd
cell_props=pd.read_csv('New_Model_Data/model_variable_all_breast_patients.csv')
cell_props.set_index('Mixture',inplace=True)
cell_props.head()

,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,T cells CD4 memory activated,T cells follicular helper,T cells regulatory (Tregs),T cells gamma delta,...,Neutrophils,Clusters,HMGB1,IL12,E,IL6,IL10,IFNG,Tumor_Size,Tumor_weight
Mixture,,,,,,,,,,,,,,,,,,,,,
TCGA-UU-A93S-01,0.048640,0.000000,0.059709,0.014494,0.0,0.016852,0.00000,0.139061,0.024382,0.000000,...,0.001748,0,2.589892,0.646120,0.136979,0.025101,0.222899,0.035531,NaN,980
TCGA-E9-A1N6-01,0.022411,0.000000,0.028556,0.002904,0.0,0.133782,0.00000,0.064145,0.017324,0.000000,...,0.001103,0,2.553876,0.013097,1.841691,0.013571,0.487788,0.046657,NaN,220
TCGA-A1-A0SP-01,0.060632,0.000000,0.041495,0.003579,0.0,0.143068,0.00000,0.061029,0.047027,0.000000,...,0.004049,0,3.032969,0.383953,0.230718,0.037220,0.290313,0.026872,NaN,40
MB-4127,0.004729,0.033260,0.000000,0.000000,0.0,0.050323,0.05312,0.068634,0.035002,0.010154,...,0.001680,0,7.225032,10.482408,11.756767,5.399032,5.286956,5.618480,40.0,NaN
MB-5058,0.030947,0.006422,0.000000,0.061530,0.0,0.000000,0.00000,0.074680,0.031758,0.105954,...,0.000000,0,7.115528,10.721697,10.944275,5.627396,5.353274,5.457621,15.0,NaN


In [2]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

In [3]:
#read meta and tcga data patients ID
meta_patients=pd.read_csv('meta_patients.csv')
meta_patients.set_index('Mixture',inplace=True)
tcga_patients=pd.read_csv('tcga_patients.csv')
tcga_patients.set_index('Mixture',inplace=True)
meta_patients=meta_patients.index
tcga_patients=tcga_patients.index


In [4]:
SS_variable_list=['Tn', 'Th', 'Tc', 'Tr', 'Dn', 'D', 'Mn', 'M', 'C', 'N', 'A','E','Ig','H','IL12','IL10','IL6']
IC_variable_list=['Tn', 'Th', 'Tc', 'Tr', 'Dn', 'D', 'Mn', 'M', 'C', 'N', 'A','E','Ig','H','IL12','IL10','IL6']
Immune_cells=['Tn', 'Th', 'Tc', 'Tr', 'Dn', 'D', 'Mn', 'M']

In [5]:
index=cell_props.index
# rearrange the data corresponding to variables
full_data=pd.DataFrame(index=index)
full_data['Sample ID']=cell_props.index
full_data['Tn']=cell_props[['T cells CD4 naive', 'T cells CD4 memory resting', 'NK cells resting']].sum(axis=1)
full_data['Th']=cell_props['T cells CD4 memory activated']+cell_props['T cells follicular helper']
full_data['Tc']=cell_props['T cells CD8']+cell_props['NK cells activated']
full_data['Tr']=cell_props['T cells regulatory (Tregs)']
full_data['Dn']=cell_props['Dendritic cells resting']
full_data['D']=cell_props['Dendritic cells activated']
full_data['M']=cell_props['Macrophages M1']+cell_props['Macrophages M2']
full_data['Mn']=cell_props[['Monocytes', 'Macrophages M0']].sum(axis=1)
full_data['C']=""
full_data['N']=""
full_data['E']=cell_props['E']
full_data['IL12']=cell_props['IL12']
full_data['IL10']=cell_props['IL10']
full_data['IL6']=cell_props['IL6']
full_data['H']=cell_props['HMGB1']
full_data['Ig']=cell_props['IFNG']
full_data['Clusters']=cell_props['Clusters']+1
full_data['Tumor_weight']=cell_props['Tumor_weight']
full_data['dim']=cell_props['Tumor_Size']

full_data['TIC']=full_data[['Tn', 'Th', 'Tc', 'Tr','Dn', 'D', 'M','Mn']].sum(axis=1)

#Substitute zeros with something negligibly small, but positive
mincellvalue=full_data[Immune_cells].to_numpy()[:]
mincellvalue=mincellvalue[mincellvalue>0].min()
for key in Immune_cells:
    full_data.loc[full_data[key]<=0, key]=0.1*mincellvalue

In [6]:
full_data_meta=full_data.drop(index=tcga_patients)
full_data_tcga=full_data.drop(index=meta_patients)



In [7]:
print(full_data_meta.shape)
full_data_meta=full_data_meta[~full_data_meta['dim'].isnull()]
print(full_data_meta.shape)


(1904, 21)
(1884, 21)


In [8]:
# Mean tumor dimension global and for each cluster
import numpy as np

full_data_meta['dim']=full_data_meta.dim.astype(float)


meandim=np.empty(6)
meandim[0]=full_data_meta['dim'].mean()
for i in range(5):
    meandim[i+1]=full_data_meta[full_data_meta['Clusters']==i+1]['dim'].mean()


meandim

array([26.23872611, 26.59764851, 25.79910714, 25.79445087, 26.73643137,
       25.30930233])

In [9]:
#0.75% quantile tumor dimension glo
Qdim=np.empty(6)
Qdim[0]=full_data_meta['dim'].quantile([0.75]) 
for i in range(5):
    Qdim[i+1]=full_data_meta[full_data_meta['Clusters']==i+1]['dim'].quantile([0.75]) 


Qdim

array([30., 30., 30., 30., 30., 29.])

In [10]:
# Mean total immune cell count global and for each cluster
meanTIC=np.empty(6)
meanTIC[0]=full_data_meta['TIC'].mean()
for i in range(5):
    meanTIC[i+1]=full_data_meta[full_data_meta['Clusters']==i+1]['TIC'].mean()
meanTIC

array([0.74841829, 0.81529441, 0.76589412, 0.79235812, 0.76605781,
       0.63197396])

In [11]:
#C:I:N ratio 0.6:0.3:0.1
# Evaluating number of cancer and necrotic cells from tumor size and immune cells
full_data_meta.C=6*((full_data_meta.dim/meandim[0])-0.3*(full_data_meta.TIC/meanTIC[0]))/7
full_data_meta.N=full_data_meta.C/6
full_data_meta[Immune_cells]*=4.5e4/meanTIC[0]
full_data_meta[['C', 'N']]*=4.5e4/0.6
full_data_meta['A']=full_data_meta[['Tn', 'Th', 'Tc', 'Tr', 'Dn', 'D', 'Mn', 'M']].sum(axis=1)*2

In [12]:
print(full_data_meta.shape)
full_data_meta=full_data_meta.loc[full_data_meta.C>=0]
print(full_data_meta.shape)

(1884, 22)
(1858, 22)


In [13]:
full_data_meta.head()

,Sample ID,Tn,Th,Tc,Tr,Dn,D,M,Mn,C,N,E,IL12,IL10,IL6,H,Ig,Clusters,Tumor_weight,dim,TIC,A
Mixture,,,,,,,,,,,,,,,,,,,,,,
MB-4127,MB-4127,8471.731848,7320.686860,0.019521,2104.557209,853.513120,0.019521,15268.716065,18707.632290,75404.057367,12567.342895,11.756767,10.482408,5.286956,5.399032,7.225032,5.618480,1,NaN,40.0,0.876927,105453.752867
MB-5058,MB-5058,0.019521,4490.293450,6739.846346,1909.497640,0.019521,0.019521,14327.085667,21693.134534,15681.967106,2613.661184,10.944275,10.721697,5.353274,5.627396,7.115528,5.457621,1,NaN,15.0,0.817603,98319.832398
MB-5060,MB-5060,3208.377992,4169.118795,0.019521,2198.367041,0.019521,0.019521,20192.131083,22836.742643,38705.908347,6450.984725,17.267685,11.023170,5.317828,5.421131,7.055549,5.181857,1,NaN,25.0,0.874897,105209.592233
MB-5062,MB-5062,4145.634294,4762.626289,2202.203776,1399.109511,0.019521,0.019521,17661.095074,15312.331429,29507.922251,4917.987042,16.910151,10.938032,5.328764,5.639162,6.712774,6.099193,1,NaN,20.0,0.756451,90966.078829
MB-5383,MB-5383,7775.699352,1802.940999,1950.346642,0.019521,0.019521,82.489960,17574.356954,22600.182385,161558.380628,26926.396771,16.673677,10.597825,5.266151,5.705890,7.011786,5.277458,1,NaN,75.0,0.861280,103572.110671


# TCGA_Data

In [14]:
print(full_data_tcga.shape)
full_data_tcga=full_data_tcga[~full_data_tcga['Tumor_weight'].isnull()]
print(full_data_tcga.shape)

(1089, 21)
(1079, 21)


In [15]:
# Mean tumor weight global and for each cluster
import numpy as np
full_data_tcga=full_data_tcga.replace('[Not Available]',np.NaN)
full_data_tcga=full_data_tcga[~full_data_tcga['Tumor_weight'].isnull()]
print(full_data_tcga.shape)

full_data_tcga['Tumor_weight']=full_data_tcga.Tumor_weight.astype(float)
full_data_tcga['Tumor_weight']/=10

meanweight=np.empty(6)
meanweight[0]=full_data_tcga['Tumor_weight'].mean()
for i in range(5):
    meanweight[i+1]=(full_data_tcga[full_data_tcga['Clusters']==i+1]['Tumor_weight'].mean())


meanweight

(1078, 21)


array([30.01883117, 26.55516014, 31.95441176, 29.57421875, 30.43902439,
       36.20833333])

In [16]:
Qweight=np.empty(6)
Qweight[0]=full_data_tcga['Tumor_weight'].quantile([0.75]) 
for i in range(5):
    Qweight[i+1]=full_data_tcga[full_data_tcga['Clusters']==i+1]['Tumor_weight'].quantile([0.75])
Qweight





array([37. , 34. , 39. , 40. , 29. , 44.5])

In [17]:
# Mean total immune cell count global and for each cluster
meanTIC=np.empty(6)
meanTIC[0]=full_data_tcga['TIC'].mean()
for i in range(5):
    meanTIC[i+1]=full_data_tcga[full_data_tcga['Clusters']==i+1]['TIC'].mean()
meanTIC

array([0.81506996, 0.86244074, 0.80294618, 0.79968757, 0.83415507,
       0.63236729])

In [18]:
necro=pd.read_csv('necro_tcga.csv')
necro.set_index('bcr_patient_barcode',inplace=True)
necro.head()

,necrosis_percent
bcr_patient_barcode,
TCGA-3C-AAAU-01,1.0
TCGA-3C-AALI-01,5.0
TCGA-3C-AALJ-01,2.0
TCGA-3C-AALK-01,2.0
TCGA-4H-AAAK-01,0.0


In [19]:
full_data_tcga['percent_necro']=necro[['necrosis_percent']]/100
print(full_data_tcga.shape)
#remove patients that necrosis percantages are unavailable
full_data_tcga=full_data_tcga[~full_data_tcga['percent_necro'].isnull()]
print(full_data_tcga.shape)
full_data_tcga['CI_percent']=1-full_data_tcga[['percent_necro']]

(1078, 22)
(1075, 22)


In [20]:
#total cell density from size of tumor:

full_data_tcga['TCD']=full_data_tcga['Tumor_weight']*(4.5e4/0.6/meanweight[0])


In [21]:
#multiply TCD with percent of necrosis to find Necrosis count:
full_data_tcga.N=full_data_tcga.TCD*full_data_tcga.percent_necro
#TCD=C+I+N
# assume C=2I so TCD*(cancer+ imuune cell percent )=C+C/2:
full_data_tcga.C=2*((full_data_tcga.TCD*full_data_tcga.CI_percent)/3)
full_data_tcga['I']=full_data_tcga.C/2
for key in Immune_cells:
    #multiply immunes ratio with calculated total immune count
    full_data_tcga[key]=full_data_tcga[key]*full_data_tcga['I']
full_data_tcga['A']=full_data_tcga[['Tn', 'Th', 'Tc', 'Tr', 'Dn', 'D', 'Mn', 'M']].sum(axis=1)*2    




In [22]:
minC=full_data_tcga.loc[full_data_tcga.C<0]
minC.shape

(0, 26)

# TCGA -IC

In [23]:
#TCGA initial conditions
IC_data_tcga=pd.DataFrame()
for i in range(5):
   

    IC_data_tcga=IC_data_tcga.append(full_data_tcga.loc[full_data_tcga.loc[(full_data_tcga['Clusters']==i+1),'Tumor_weight'].idxmin(),['Sample ID']+ IC_variable_list], ignore_index=True)
IC_data_tcga.drop(columns='Sample ID',inplace=True)
IC_data_tcga

,A,C,D,Dn,E,H,IL10,IL12,IL6,Ig,M,Mn,N,Tc,Th,Tn,Tr
0,1588.168849,1665.621147,0.000270,0.000270,0.325040,3.220856,0.198742,0.681820,0.016630,0.000000,151.483203,486.815638,0.000000,0.000270,60.054545,55.283403,40.446824
1,579.873481,832.810573,0.000135,0.000135,0.724515,2.157618,0.516651,2.012798,0.058181,0.324038,58.908808,42.423253,0.000000,33.925737,59.899509,58.826129,35.953034
2,2245.411843,3331.242294,0.052870,6.422324,3.718043,2.649092,0.188855,0.022534,0.000556,0.020070,769.227164,19.514870,0.000000,66.635939,57.269717,166.174899,37.408139
3,10523.969011,13324.969175,88.714596,0.002163,0.338669,2.467550,1.293488,2.735730,0.113913,1.286300,1890.978497,901.419427,4996.863441,432.851189,1007.640483,781.630012,158.748138
4,4232.752401,6662.484588,0.001082,46.081812,2.271700,3.351141,0.061684,0.149375,0.007705,0.016857,1050.531541,213.101109,0.000000,63.696787,223.709827,473.280996,45.973047


# Meta -IC

In [26]:

IC_data_meta=pd.DataFrame()
for i in range(5):
   
    IC_data_meta=IC_data_meta.append(full_data_meta.loc[full_data_meta.loc[(full_data_meta['Clusters']==i+1),'dim'].idxmin(), ['Sample ID']+IC_variable_list+[]]
                          , ignore_index=True)
IC_data_meta.drop(columns='Sample ID',inplace=True)

IC_data_meta

,A,C,D,Dn,E,H,IL10,IL12,IL6,Ig,M,Mn,N,Tc,Th,Tn,Tr
0,96152.087961,1446.284397,0.019521,0.019521,14.912336,6.905254,5.611189,10.966744,7.318836,6.092462,10593.964752,26254.600402,241.047399,5881.781147,3807.026341,469.646246,1068.986051
1,96655.922852,3788.343246,0.019521,2132.149997,16.790074,7.856585,5.337778,10.796108,5.987496,5.531628,22368.363360,5223.602084,631.390541,5859.416730,4267.453780,7504.110992,972.844962
2,85493.838532,3730.194989,0.019521,0.019521,14.593802,7.921497,5.386976,10.467842,8.534527,5.607182,18564.591204,13489.456747,621.699165,2716.708138,2244.915289,5695.072327,36.136520
3,79345.684806,2597.607934,2597.159150,702.222903,12.423053,8.111669,5.253752,11.031914,6.695582,6.911703,7479.954616,3686.873754,432.934656,5084.315474,8182.720746,8782.433641,3157.162119
4,70554.303320,4481.483762,0.019521,985.353311,14.985584,8.971442,5.253552,11.194911,6.058305,5.381490,11080.653011,5486.521692,746.913960,6153.883726,2434.606188,7078.748318,2057.365895


In [27]:
#get average of meta and tcga IC data

IC_data=(IC_data_meta+IC_data_tcga)/2
IC_data

    

,A,C,D,Dn,E,H,IL10,IL12,IL6,Ig,M,Mn,N,Tc,Th,Tn,Tr
0,48870.128405,1555.952772,0.009896,0.009896,7.618688,5.063055,2.904965,5.824282,3.667733,3.046231,5372.723977,13370.708020,120.523700,2940.890709,1933.540443,262.464824,554.716438
1,48617.898166,2310.576910,0.009828,1066.075066,8.757294,5.007102,2.927214,6.404453,3.022839,2.927833,11213.636084,2633.012668,315.695270,2946.671233,2163.676644,3781.468561,504.398998
2,43869.625188,3530.718641,0.036195,3.220923,9.155923,5.285294,2.787916,5.245188,4.267542,2.813626,9666.909184,6754.485808,310.849582,1391.672038,1151.092503,2930.623613,36.772329
3,44934.826909,7961.288555,1342.936873,351.112533,6.380861,5.289610,3.273620,6.883822,3.404748,4.099002,4685.466556,2294.146591,2714.899048,2758.583332,4595.180615,4782.031826,1657.955128
4,37393.527861,5571.984175,0.010301,515.717561,8.628642,6.161291,2.657618,5.672143,3.033005,2.699174,6065.592276,2849.811400,373.456980,3108.790257,1329.158007,3776.014657,1051.669471


In [28]:
full_data=pd.concat([full_data_tcga,full_data_meta],axis=0)


In [29]:
max_variable=full_data[IC_variable_list].max()
max_variable

Tn       42704.744707
Th       15987.511066
Tc       24949.826521
Tr       11735.530761
Dn        6506.502280
D         8273.739237
Mn       45767.452797
M        52470.572891
C       425754.401784
N       101936.014190
A       261167.279742
E           18.680700
Ig           8.768142
H           11.595535
IL12        15.159334
IL10         5.962510
IL6         10.734240
dtype: float64

In [30]:
mean_variable=full_data[IC_variable_list].mean()
mean_variable

Tn       4402.978812
Th       3729.677968
Tc       4129.095508
Tr       1163.230463
Dn        389.473652
D         262.750754
Mn       8521.345255
M       12961.494938
C       46375.888988
N        6309.182087
A       71120.094702
E          10.325422
Ig          3.675418
H           5.993292
IL12        7.155235
IL10        3.505264
IL6         3.906353
dtype: float64

In [31]:
# Evaluating averages among patients with large tumors for steady state conditions
meta_Large_tumor=pd.DataFrame()
for i in range(5):
    meta_Large_tumor=meta_Large_tumor.append(full_data_meta[(full_data_meta['Clusters']==i+1) & 
                                                         (full_data_meta['dim']>Qdim[i+1])])

meta_Large_tumor.head()

,Sample ID,Tn,Th,Tc,Tr,Dn,D,M,Mn,C,N,E,IL12,IL10,IL6,H,Ig,Clusters,Tumor_weight,dim,TIC,A
Mixture,,,,,,,,,,,,,,,,,,,,,,
MB-4127,MB-4127,8471.731848,7320.686860,0.019521,2104.557209,853.513120,0.019521,15268.716065,18707.632290,75404.057367,12567.342895,11.756767,10.482408,5.286956,5.399032,7.225032,5.618480,1,NaN,40.0,0.876927,105453.752867
MB-5383,MB-5383,7775.699352,1802.940999,1950.346642,0.019521,0.019521,82.489960,17574.356954,22600.182385,161558.380628,26926.396771,16.673677,10.597825,5.266151,5.705890,7.011786,5.277458,1,NaN,75.0,0.861280,103572.110671
MB-5490,MB-5490,6153.911808,4343.532830,2182.199277,2011.130092,0.019521,0.019521,16052.238386,17689.066570,77244.668122,12874.111354,17.368607,10.598834,5.211557,5.453162,8.366079,5.505102,1,NaN,40.0,0.805499,96864.236012
MB-4008,MB-4008,5546.799799,4524.755492,5648.324213,1132.454951,0.019521,0.019521,17503.472001,14992.770195,64601.724225,10766.954038,14.991440,10.733096,5.201949,5.450493,7.576894,5.389781,1,NaN,35.0,0.820742,98697.231384
MB-5377,MB-5377,2405.766273,3526.962215,1165.434238,3027.519726,0.019521,0.019521,17743.727257,21354.600283,89155.428284,14859.238047,17.084616,10.664877,5.296277,5.515432,7.995505,5.296126,1,NaN,45.0,0.818670,98448.098070


In [32]:
# Evaluating averages among patients with large tumors for steady state conditions
tcga_Large_tumor=pd.DataFrame()
for i in range(5):
    tcga_Large_tumor=tcga_Large_tumor.append(full_data_tcga[(full_data_tcga['Clusters']==i+1) & 
                                                         ((full_data_tcga['Tumor_weight'])>Qweight[i+1])])


tcga_Large_tumor.head()

,Sample ID,Tn,Th,Tc,Tr,Dn,D,M,Mn,C,N,E,IL12,IL10,IL6,H,Ig,Clusters,Tumor_weight,dim,TIC,percent_necro,CI_percent,TCD,I,A
Mixture,,,,,,,,,,,,,,,,,,,,,,,,,,
TCGA-UU-A93S-01,TCGA-UU-A93S-01,4570.937364,10782.013073,3898.539507,1890.430539,0.025173,1244.619979,18551.907901,24044.934382,155069.328776,12242.31543,0.136979,0.646120,0.222899,0.025101,2.589892,0.035531,1,98.0,NaN,0.838120,0.05,0.95,244846.308594,77534.664388,129966.815835
TCGA-AR-A24S-01,TCGA-AR-A24S-01,8802.647873,2166.080327,0.014060,814.178970,2228.875774,0.014060,12335.101136,11733.525610,86612.299639,0.00000,2.380458,0.698897,1.143689,0.073564,2.823753,0.028789,1,52.0,NaN,0.879330,0.00,1.00,129918.449458,43306.149819,76160.875622
TCGA-AR-A5QM-01,TCGA-AR-A5QM-01,4281.391339,1480.520826,827.142678,490.026891,0.009463,360.539997,5571.742241,8006.217267,58296.740141,0.00000,2.632093,2.913581,0.914190,0.328826,1.369317,0.065581,1,35.0,NaN,0.721055,0.00,1.00,87445.110212,29148.370071,42035.181405
TCGA-AO-A0JM-01,TCGA-AO-A0JM-01,2995.427424,6261.560916,289.999389,4744.660431,0.019197,417.530352,12493.419971,24833.057107,118259.101430,0.00000,0.728003,0.665964,0.321573,0.328964,2.239460,0.066865,1,71.0,NaN,0.880028,0.00,1.00,177388.652145,59129.550715,104071.349576
TCGA-AR-A24Q-01,TCGA-AR-A24Q-01,7888.145024,3633.223680,3399.282333,1006.097211,0.015953,0.015953,11372.805690,12141.048769,98271.647667,0.00000,0.141024,1.466783,0.194291,0.040392,2.410734,0.257318,1,59.0,NaN,0.802685,0.00,1.00,147407.471501,49135.823834,78881.269222


In [33]:
tcga_mean=pd.DataFrame()
for i in range(5):
    tcga_mean=tcga_mean.append(tcga_Large_tumor[tcga_Large_tumor['Clusters']==i+1].mean(), ignore_index=True)
tcga_mean[SS_variable_list]

,Tn,Th,Tc,Tr,Dn,D,Mn,M,C,N,A,E,Ig,H,IL12,IL10,IL6
0,5034.328848,3092.452256,1322.493322,1651.176152,117.852834,396.983590,15469.035611,11451.710522,88467.087759,6390.631422,77072.066272,1.511554,0.107295,2.631091,0.559251,0.294373,0.082552
1,11279.504845,4865.131961,3794.283115,1808.364241,236.217242,653.100717,6721.981738,16115.962676,113843.582116,8719.420838,90949.093070,1.785085,0.274476,2.933317,1.143590,0.510122,0.159575
2,6670.042851,3246.354232,2200.881133,1486.242778,330.890195,423.358314,7784.007536,19664.576961,104219.280425,4881.362171,83612.707997,2.919157,0.055086,2.683120,0.305914,0.333980,0.110898
3,5005.079194,7702.810916,8212.292423,2622.529749,69.312848,449.585625,7391.293710,14506.379942,104675.960977,8881.924766,91918.568815,1.211786,2.638844,2.925938,1.874508,0.876060,0.195985
4,9958.282440,5091.225160,4786.962572,1629.591305,426.283179,833.852302,5309.753068,17016.905885,141605.557839,1207.575332,90105.711821,1.276599,0.051073,2.688918,0.781002,0.368164,0.112351


In [34]:
meta_mean=pd.DataFrame()
for i in range(5):
    meta_mean=meta_mean.append(meta_Large_tumor[meta_Large_tumor['Clusters']==i+1].mean(), ignore_index=True)


In [35]:
large_all=pd.concat([tcga_Large_tumor,meta_Large_tumor],axis=0)
large_all=large_all.drop(columns='Sample ID')


In [36]:
Large_tumor_means=pd.DataFrame()
for i in range(5):
    Large_tumor_means=Large_tumor_means.append(large_all[large_all['Clusters']==i+1].mean(), ignore_index=True)

In [38]:
Large_tumor_means[SS_variable_list]

,Tn,Th,Tc,Tr,Dn,D,Mn,M,C,N,A,E,Ig,H,IL12,IL10,IL6
0,4547.373831,3866.006181,2443.741216,1920.893798,125.994505,327.740040,17982.476160,13856.322163,90318.704826,11509.045808,90141.095787,8.863229,3.210116,5.610406,6.424776,3.171261,3.415329
1,11262.540634,4770.533462,4022.093507,1545.945965,327.139233,609.988673,6955.550896,16224.200643,111760.038633,10164.910010,91435.986026,4.027236,1.192018,3.842605,2.836787,1.356432,1.283688
2,5725.293876,3699.588047,2785.752865,1514.088780,400.274321,351.954687,8768.844812,20743.675699,96786.420186,8112.126958,87978.946174,7.679039,1.967611,4.491128,4.015819,2.126792,2.138444
3,4136.300350,5953.843490,7708.289419,1656.810827,499.115380,437.151055,9808.888848,15892.237443,95428.308138,15444.284220,92185.273621,13.703616,5.716169,7.539870,10.434455,5.076178,5.797373
4,5686.780974,3908.200423,5555.196677,616.486014,703.592587,222.399170,6452.652711,15190.802159,99041.803422,15285.511729,76672.221429,15.046261,5.165006,7.697891,10.187633,5.045726,6.006308
